In [91]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from cnocr import CnOcr
import numpy as np
import cv2

In [92]:
# 自動下載ChromeDriver
service = ChromeService(executable_path=ChromeDriverManager().install())

# 關閉通知提醒 pip install webdriver-manager --upgrade
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('blink-settings=imagesEnabled=false') 
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

url = 'https://www.indievox.com/activity/detail/23_iv02706f5'
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)

In [93]:
C = driver.find_element(By.XPATH, value='//*[@id="content"]/div[1]/div[2]/ul/li/a')
C.click()

In [94]:
time.sleep(2)
D = driver.find_element(By.XPATH, value='//*[@id="gameList"]/table/tbody/tr/td[4]/button')
D.click()

In [95]:
ocr = CnOcr(det_model_name='en_PP-OCRv3_det', rec_model_name='en_PP-OCRv3') 

# 使用 XPath 找到圖片元素
image_element = driver.find_element(By.XPATH, value='//*[@id="TicketForm_verifyCode-image"]')

# 獲取圖片的 src 屬性
image_src = image_element.get_attribute("src")

time.sleep(2)
# 下載圖片
driver.get(image_src)

# 將圖片內容轉換為 NumPy 陣列
screenshot = driver.get_screenshot_as_png()
image_content = np.frombuffer(screenshot, dtype=np.uint8)
image = cv2.imdecode(image_content, cv2.IMREAD_COLOR)
image_np = np.array(image)
out = ocr.ocr(image)
print(out[0]['text'])
driver.back()

Umie


In [96]:
from selenium.webdriver.support.ui import Select
# time.sleep(1)
dropdown = driver.find_element(By.XPATH, '//*[@id="TicketForm_ticketPrice_01"]')
select = Select(dropdown)
# 選擇下拉框中的選項（根據索引）
select.select_by_index(1)  # 選擇第二個選項（索引從 0 開始）

In [97]:
from selenium.webdriver.common.keys import Keys

check_box = driver.find_element(By.XPATH, value = '//*[@id="TicketForm_agree"]')
check_box.click()

# 定位到輸入框元素
search_box = driver.find_element(By.XPATH, value = '//*[@id="TicketForm_verifyCode"]')  # 假設這是搜尋框的名字

# 在輸入框中輸入文字
search_box.send_keys(out[0]['text'])

# 也可以模擬按下 Enter 鍵
search_box.send_keys(Keys.RETURN)